In [1]:
import torch

def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    Calcula la Intersection Over Union (IoU) entre bounding boxes.

    Args:
        boxes_preds (tensor): Bounding boxes predichas de forma (N, 4) o (batch_size, 4)
                            donde N es el número de cajas o 1 si es una sola caja.
                            Formato de las cajas: (x, y, w, h) o (x1, y1, x2, y2).
        boxes_labels (tensor): Bounding boxes ground truth de forma (N, 4) o (batch_size, 4).
        box_format (str): Formato de las cajas de entrada.
                        "midpoint" si es (x_center, y_center, width, height)
                        "corners" si es (x1, y1, x2, y2)

    Returns:
        tensor: IoU para cada par de cajas, de forma (N, 1) o (batch_size, 1).
    """

    if box_format == "midpoint":
        # Convertir de (x_center, y_center, width, height) a (x1, y1, x2, y2)
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    elif box_format == "corners":
        # Asumir que ya están en (x1, y1, x2, y2)
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]
        
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]
    else:
        raise ValueError("box_format debe ser 'midpoint' o 'corners'")

    # Calcular las coordenadas del rectángulo de intersección
    x1_inter = torch.max(box1_x1, box2_x1)
    y1_inter = torch.max(box1_y1, box2_y1)
    x2_inter = torch.min(box1_x2, box2_x2)
    y2_inter = torch.min(box1_y2, box2_y2)

    # Calcular el área de intersección
    # torch.clamp asegura que el ancho/alto no sea negativo si no hay intersección
    intersection = (x2_inter - x1_inter).clamp(0) * \
                (y2_inter - y1_inter).clamp(0)

    # Calcular el área de cada bounding box
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    # Calcular el IoU
    union = box1_area + box2_area - intersection + 1e-6 # Añadir epsilon para evitar división por cero
    iou = intersection / union

    return iou



In [2]:
# Ejemplo de uso de la función intersection_over_union

import torch
# Asegúrate de que la función intersection_over_union esté accesible,
# por ejemplo, si la tienes en un archivo llamado 'utils.py', la importarías así:
# from utils import intersection_over_union

# --- Ejemplo 1: Cajas superpuestas (formato 'corners') ---
print("--- Ejemplo 1: Cajas superpuestas (formato 'corners') ---")
box1 = torch.tensor([[0.0, 0.0, 10.0, 10.0]]) # Caja 1: (0,0) a (10,10)
box2 = torch.tensor([[5.0, 5.0, 15.0, 15.0]]) # Caja 2: (5,5) a (15,15)
# Intersección: (5,5) a (10,10) -> Ancho 5, Alto 5 -> Área = 25
# Área box1 = 100, Área box2 = 100
# Unión = 100 + 100 - 25 = 175
# IoU esperado = 25 / 175 = 0.1428...

iou_result = intersection_over_union(box1, box2, box_format="corners")
print(f"IoU (corners): {iou_result.item()}")
# Debería ser aproximadamente 0.142857

# --- Ejemplo 2: Cajas idénticas (formato 'midpoint') ---
print("\n--- Ejemplo 2: Cajas idénticas (formato 'midpoint') ---")
# Caja 1: centro (5,5), ancho 10, alto 10 -> (0,0) a (10,10)
box1_mid = torch.tensor([[5.0, 5.0, 10.0, 10.0]])
# Caja 2: centro (5,5), ancho 10, alto 10 -> (0,0) a (10,10)
box2_mid = torch.tensor([[5.0, 5.0, 10.0, 10.0]])
# IoU esperado = 1.0 (cajas idénticas)

iou_result_mid = intersection_over_union(box1_mid, box2_mid, box_format="midpoint")
print(f"IoU (midpoint, idénticas): {iou_result_mid.item()}")
# Debería ser 1.0

# --- Ejemplo 3: Cajas sin superposición (formato 'corners') ---
print("\n--- Ejemplo 3: Cajas sin superposición (formato 'corners') ---")
box1_no_overlap = torch.tensor([[0.0, 0.0, 10.0, 10.0]])
box2_no_overlap = torch.tensor([[11.0, 11.0, 20.0, 20.0]])
# IoU esperado = 0.0

iou_result_no_overlap = intersection_over_union(box1_no_overlap, box2_no_overlap, box_format="corners")
print(f"IoU (corners, sin superposición): {iou_result_no_overlap.item()}")
# Debería ser 0.0

# --- Ejemplo 4: Cajas con una dimensión cero (para probar clamp) ---
print("\n--- Ejemplo 4: Cajas con una dimensión cero (para probar clamp) ---")
box1_zero_width = torch.tensor([[0.0, 0.0, 0.0, 10.0]]) # Ancho cero
box2_zero_width = torch.tensor([[0.0, 0.0, 5.0, 10.0]])
# IoU esperado = 0.0 (o un valor muy pequeño debido al epsilon)

iou_result_zero_width = intersection_over_union(box1_zero_width, box2_zero_width, box_format="corners")
print(f"IoU (corners, ancho cero): {iou_result_zero_width.item()}")
# Debería ser 0.0 (o cercano a cero)

# --- Ejemplo 5: Cajas con diferentes tamaños (formato 'midpoint') ---
print("\n--- Ejemplo 5: Cajas con diferentes tamaños (formato 'midpoint') ---")
# box1: centro (10,10), w=10, h=10 -> (5,5) a (15,15)
box1_diff = torch.tensor([[10.0, 10.0, 10.0, 10.0]])
# box2: centro (10,10), w=5, h=5 -> (7.5,7.5) a (12.5,12.5)
box2_diff = torch.tensor([[10.0, 10.0, 5.0, 5.0]])
# box1_area = 100, box2_area = 25
# Intersección = 25 (box2 está completamente dentro de box1)
# Unión = 100 + 25 - 25 = 100
# IoU esperado = 25 / 100 = 0.25

iou_result_diff = intersection_over_union(box1_diff, box2_diff, box_format="midpoint")
print(f"IoU (midpoint, diferentes tamaños): {iou_result_diff.item()}")
# Debería ser 0.25

--- Ejemplo 1: Cajas superpuestas (formato 'corners') ---
IoU (corners): 0.1428571492433548

--- Ejemplo 2: Cajas idénticas (formato 'midpoint') ---
IoU (midpoint, idénticas): 1.0

--- Ejemplo 3: Cajas sin superposición (formato 'corners') ---
IoU (corners, sin superposición): 0.0

--- Ejemplo 4: Cajas con una dimensión cero (para probar clamp) ---
IoU (corners, ancho cero): 0.0

--- Ejemplo 5: Cajas con diferentes tamaños (formato 'midpoint') ---
IoU (midpoint, diferentes tamaños): 0.25
